In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

import openai

In [2]:
load_dotenv() # Loads the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
df = pd.read_csv("outputs/summaries1.csv", usecols=["input_excerpts", "summaries_llm", "summaries_inhouse", "tag_name"])

In [4]:
class SummEvaluation:
    def __init__(self):
        pass

    def get_geval_score(
        self,
        criteria: str,
        steps: str,
        document: str,
        summary: str,
        metric_name: str
    ):
        prompt = self.prompt.format(
            criteria=criteria,
            steps=steps,
            metric_name=metric_name,
            document=document,
            summary=summary,
        )
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=5,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response.choices[0].message.content.strip()

    def eval_summ_using_prompt(self, df):
        self.prompt = """
            You will be given one summary written for an article. Your task is to rate the summary on one metric.
            Please make sure you read and understand these instructions very carefully. 
            Please keep this document open while reviewing, and refer to it as needed.

            Evaluation Criteria:

            {criteria}

            Evaluation Steps:

            {steps}

            Example:

            Source Text:

            {document}

            Summary:

            {summary}

            Evaluation Form (scores ONLY):

            - {metric_name}
        """

        # Metric 1: Relevance

        RELEVANCY_SCORE_CRITERIA = """
        Relevance(1-5) - selection of important content from the source. \
        The summary should include only important information from the source document. \
        Annotators were instructed to penalize summaries which contained redundancies and excess information.
        """

        RELEVANCY_SCORE_STEPS = """
        1. Read the summary and the source document carefully.
        2. Compare the summary to the source document and identify the main points of the article.
        3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
        4. Assign a relevance score from 1 to 5.
        """

        # Metric 2: Coherence

        COHERENCE_SCORE_CRITERIA = """
        Coherence(1-5) - the collective quality of all sentences. \
        We align this dimension with the DUC quality question of structure and coherence \
        whereby "the summary should be well-structured and well-organized. \
        The summary should not just be a heap of related information, but should build from sentence to a\
        coherent body of information about a topic."
        """

        COHERENCE_SCORE_STEPS = """
        1. Read the article carefully and identify the main topic and key points.
        2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
        and if it presents them in a clear and logical order.
        3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
        """

        # Metric 3: Consistency

        CONSISTENCY_SCORE_CRITERIA = """
        Consistency(1-5) - the factual alignment between the summary and the summarized source. \
        A factually consistent summary contains only statements that are entailed by the source document. \
        Annotators were also asked to penalize summaries that contained hallucinated facts.
        """

        CONSISTENCY_SCORE_STEPS = """
        1. Read the article carefully and identify the main facts and details it presents.
        2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
        3. Assign a score for consistency based on the Evaluation Criteria.
        """

        # Metric 4: Fluency

        FLUENCY_SCORE_CRITERIA = """
        Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
        1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
        2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
        3: Good. The summary has few or no errors and is easy to read and follow.
        """

        FLUENCY_SCORE_STEPS = """
        Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
        """

        self.evaluation_metrics = {
            "relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
            "coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
            "consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
            "fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS)
        }
        results = {}
        for eval_type in self.evaluation_metrics.keys():
            results.update({
                f"{eval_type}_llm": [],
                f"{eval_type}_inhouse": []
            })
        for eval_type, (criteria, steps) in self.evaluation_metrics.items():
            results[f"{eval_type}_llm"] = df.apply(lambda x: self.get_geval_score(criteria, steps, x["input_excerpts"], x["summaries_llm"], eval_type), axis=1).tolist()
            results[f"{eval_type}_inhouse"] = df.apply(lambda x: self.get_geval_score(criteria, steps, x["input_excerpts"], x["summaries_inhouse"], eval_type), axis=1).tolist()
        
        return results
    

In [5]:
df

,input_excerpts,summaries_llm,summaries_inhouse,tag_name
0,"Current needs include WASH, shelter, health, a...",The earthquake in Morocco has caused significa...,Moroccan authorities are continuing their effo...,protection
1,"Current needs include WASH, shelter, health, a...",The earthquake in Morocco has caused extensive...,The devastating earthquake in Morocco has left...,shelter
2,According to an Unosat analysis performed on 1...,"According to an Unosat analysis on 10/9/2023, ...",The earthquake that struck Morocco last week h...,education
3,"Current needs include WASH, shelter, health, a...",Caritas Morocco continues to provide aid to th...,"The earthquake in Morocco, which killed at lea...",food
4,The average standard of living of households i...,The average standard of living of households i...,The Moroccan government has announced that it ...,livelihoods
5,They are among 60 firefighters from 14 UK fire...,The earthquake in Morocco has caused significa...,The devastating earthquake that struck Morocco...,health
6,They are among 60 firefighters from 14 UK fire...,The 6.8 magnitude earthquake in Morocco's High...,Emergency supplies are being sought to help th...,cross
7,All international airports and ports are funct...,All international airports and ports are funct...,The Moroccan government has confirmed that it ...,logistics
8,"Through these completed activities, AFH has ma...","Through completed activities, AFH has made a m...",The British Farmers' Fund (AFH) has completed ...,nutrition
9,"In recent years, the Moroccan government and d...",Summary 1: This article discusses aid programs...,Moroccan aid agency CARE Maroc has launched a ...,agriculture


In [6]:
summ_prompt = SummEvaluation()
results = summ_prompt.eval_summ_using_prompt(df)

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [ ]:
df_scores = pd.DataFrame.from_dict(results)

In [ ]:
df_merged = pd.concat([df, df_scores], axis=1)

In [ ]:
df_merged